<a href="https://colab.research.google.com/github/rafpank/Colab/blob/main/data_science_bootcamp%20/uczenie_maszynowe/02_metryki_klasyfikacjia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Metryki - Problem klasyfikacji:

In [12]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#Accuracy - Dokładność klasyfikacji

Klasyfikacja BINARNA
Accuracy = (correct predictions/ total predictions) * 100

Trzeba pamiętać, że możemy oczekiwać różnych rezultatów, bo np. kiedy szukamy terrorysty na lotnisku to wolimy więcej osób mieć podejrzanych, niż przepuścić jednego terrorystę. A to oznacza że model może działąć np w 90% ale być dla nas nieodpowiedni.


In [13]:
y_true = np.array([1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1])
y_pred = np.array([0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1])

In [14]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true, y_pred)

0.8

In [15]:
result = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred}) #zrobimy prostą wizualizację naszych predykcji. Żeby to zrobić potrzebujemy DataFrame
result

,y_true,y_pred
0,1,0
1,0,0
2,1,1
3,1,1
4,0,0
5,1,1
6,1,0
7,0,0
8,1,1
9,0,1


In [16]:
result = result.sort_values(by='y_true') #sortowanie po y_true
result = result.reset_index(drop=True) #resetowanie indexu. Gdybym tego nie zrobił to index byłby zgodny z wcześniejszym wyświetleniem
result['sample'] = result.index +1 #tworzymy nową kolumnę z nazwą próbki.
result

,y_true,y_pred,sample
0,0,0,1
1,0,0,2
2,0,0,3
3,0,0,4
4,0,0,5
5,0,0,6
6,0,0,7
7,0,0,8
8,0,1,9
9,0,1,10


In [17]:
fig =make_subplots(rows=2, cols=1) #tworzymy obiekt fig i wykorzystujemy do tego funkcje 'make_subplots', ktora umożliwi nam stworzenie subploty (2 wiersze i jedna kolumna)
fig.add_trace(go.Scatter(x=result['sample'], y=result['y_true'], mode='markers', name='y_true'), row=1, col=1) #ślad wykresu (wykres punktowy), oraz w którym miejscu chcemy dodać wykres
fig.add_trace(go.Scatter(x=result['sample'], y=result['y_pred'], mode='markers', name='y_pred'), row=2, col=1)
fig.update_layout(height=500, width=900, title_text='Klasyfikator Binarny (Accuracy)')
fig.show()

Na tym wykresie widać na górnym wykresie rzeczywiste wyniki (y_true), a na drugim przewidywanie. Dzięki temu, że zastosowałem sortowanie (`result = result.sort_values(by='y_true')`) wyniki są lepiej widoczne.
Wykres ten pokazuje też, czy nas wykres lepiej przewiduje klasę 0 czy 1.

#Macierz konfuzji/pomyłek


In [24]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
cm

array([[ 8,  2],
       [ 4, 16]])

Poniewaz jest to klasyfikator binarny to dostajemy macierz 2x2. W innych przypadkach zawsze wychodzi matrix liczba klas x liczba klas. Oczywiście najważniejsze są wartości na przekątnej, które powinny być jak najwyższe

In [25]:
import plotly.figure_factory as ff #nie jest to zbyt popularny wykres dlatego jest w module figure_factory

def plot_confusion_matrix(cm):
  cm = cm[::-1] #zamieniamy kolejność macierzy konfuzj, bo to jest specyfika tej macierzy
  cm = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_1', 'true_0']) #jako obiekt DataFrame przekazuję obiekty pred i true

  fig = ff.create_annotated_heatmap(z=cm.values, x=list(cm.columns), y=list(cm.index),
                                    colorscale='ice', showscale=True, reversescale=True)
  fig.update_layout(width=500, height=500, title='Confusion Matrix', font_size=16)
  fig.show()

plot_confusion_matrix(cm)

teraz dowiemy się jakie metryki na podstawie macierzy konfuzji możemy wyznaczyc

In [26]:
cm

array([[ 8,  2],
       [ 4, 16]])

In [27]:
cm.ravel()

array([ 8,  2,  4, 16])

In [28]:
tn, fp, fn, tp = cm.ravel() #true_positive, false_positive... - przypisujemy nazwy zmiennych
print(f'TN - True Negative: {tn}')
print(f'FP - False Positive: {fp}')
print(f'FN - False Negative: {fn}')
print(f'TP - True Positive: {tp}')

TN - True Negative: 8
FP - False Positive: 2
FN - False Negative: 4
TP - True Positive: 16


In [30]:
# Błąd pierwszego rodzaju - Type I error/ False Positive Rate
fpr = fp / (fp + tn) #czyli % fałszywie pozytywnych do wszystkich negatywnych (fp+tn)
print(f'Błąd pierwszego rodzaju: {fpr}')

Błąd pierwszego rodzaju: 0.2


In [31]:
# Błąd 2 rodzaju - Type II error/ False Negative Rate. Ten błąd jest groźniejszy w skutkach
fnr = fn / (fn + tp)
print(f'Błąd drugiego rodzaju: {fnr}')

Błąd drugiego rodzaju: 0.2


In [32]:
# Precision - ile obserwacji przewidywanych jako pozytywne są w rzeczywistości pozytywne
precision = tp / (tp + fp)
print(f'Precision: {precision}')

Precision: 0.8888888888888888


In [33]:
# Recall - jak wiele obserwacji z wszystkich pozytywnych sklasyfikowanych jako pozytywne
recall = tp / (tp + fn)
print(f'Recall: {recall}')

Recall: 0.8


In [34]:
# F1 score - to średnia harmoniczna metryk precision i recall. Znajduje się to też, w raporcie klasyfikacji który stworzymy na sam koniec
f1_score = 2 * precision * recall / (precision + recall)
print(f'F1 score: {f1_score}')

F1 score: 0.8421052631578948
